In [1]:
import mne.time_frequency
import mne
from ieeg.io import get_data, raw_from_layout
from ieeg.navigate import trial_ieeg, channel_outlier_marker, crop_empty_data, outliers_to_nan
from ieeg.calc.scaling import rescale
import os
from ieeg.timefreq.utils import wavelet_scaleogram, crop_pad
import numpy as np


In [2]:
HOME = os.path.expanduser("~")

# get box directory depending on OS
if os.name == 'nt': # windows
    LAB_root = os.path.join(HOME, "Box", "CoganLab")
else: # mac
    LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

layout = get_data("GlobalLocal", root=LAB_root)
subjects = layout.get(return_type="id", target="subject")

print(subjects)
# for sub in subjects:

sub='D0057' #why does this not work in the for loop???
# Load the data
filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                           extension='.edf', desc='clean', preload=False) #get line-noise filtered data
print(filt)

## Crop raw data to minimize processing time
good = crop_empty_data(filt)

# good.drop_channels(good.info['bads'])
good.info['bads'] = channel_outlier_marker(good, 3, 2)
good.drop_channels(good.info['bads'])
# good.info['bads'] += channel_outlier_marker(good, 4, 2)
# good.drop_channels(good.info['bads'])
good.load_data()

ch_type = filt.get_channel_types(only_data_chs=True)[0]
good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

# Remove intermediates from mem
# good.plot()

## epoching and trial outlier removal

save_dir = os.path.join(layout.root, 'derivatives', 'spec', 'wavelet', sub)
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

  



['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077']
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_d

c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-02_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-03_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.
Extracting EDF parameters from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 228 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


Reading events from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_events.tsv.
Reading channel info from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_task-GlobalLocal_acq-01_run-04_desc-clean_channels.tsv.
Reading electrode coords from C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\clean\sub-D0057\ieeg\sub-D0057_acq-01_space-ACPC_electrodes.tsv.


c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Omitted 226 annotation(s) that were outside data range.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: The number of channels in the channels.tsv sidecar file (179) does not match the number of channels in the raw data file (178). Will not try to set channel names.
  new_raw = read_raw_bids(bids_path=BIDS_path)
c:\Users\jz421\Desktop\GlobalLocal\ieeg\io.py:100: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: Trigger
  new_raw = read_raw_bids(bids_path=BIDS_path)


<RawEDF | sub-D0057_task-GlobalLocal_acq-01_run-01_desc-clean_ieeg.edf, 178 x 8243200 (4025.0 s), ~201 kB, data not loaded>
outlier round 1 channels: ['RAMT8']
outlier round 2 channels: ['RAMT8', 'RPI16']
Reading 0 ... 3219820  =      0.000 ...  1572.178 secs...
Applying average reference.
Applying a custom ('sEEG',) reference.


In [10]:
trialTypes = ["c25s25"]

epochsTFRshape = 0,176,46,50

allBaselines = np.empty(shape=(epochsTFRshape))
# allSpec = np.empty(shape=(epochsTFRshape))
biggestTimeDim = 410  # 410 for D0057, need a more efficient way of finding this though.

for trialType in trialTypes:
    for epoch, t, name in zip(
            ("Stimulus"+trialType, "Stimulus"+trialType, "Response"+trialType),
            ((-0.5, 0), (-0.5, 2), (-0.5, 2)),
            ("baselineStimulus"+trialType, "Stimulus"+trialType, "Response"+trialType)):
        times = [None, None]
        times[0] = t[0] - 0.5
        times[1] = t[1] + 0.5
        trials = trial_ieeg(good, epoch, times, preload=True)
        outliers_to_nan(trials, outliers=10)
        spec = wavelet_scaleogram(trials, n_jobs=-2, decim=int(good.info['sfreq'] / 100))
        
        timeDim = spec.data.shape[3]
        # if timeDim > biggestTimeDim:
        #     biggestTimeDim = timeDim

        crop_pad(spec, "0.5s")
        print(name + ": " + str(spec.data.shape))
        if name == "baselineStimulus"+trialType:
            base = spec.copy()
            continue
        allBaselines = np.concatenate([allBaselines, base.data], axis=0)




   

Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 3073 original time points ...


0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:    9.4s


Not setting metadata
baselineStimulusc25s25: (20, 176, 46, 50)
Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
20 matching events found


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   11.0s finished


...

Stimulusc25s25: (20, 176, 46, 255)
Used Annotations descriptions: ['Responsec25n25', 'Responsec25r25', 'Responsec25r75', 'Responsec25s25', 'Responsec25s75', 'Responsec75r25', 'Responsec75r75', 'Responsec75s25', 'Responsec75s75', 'Responsei25r25', 'Responsei25r75', 'Responsei25s25', 'Responsei25s75', 'Responsei75n25', 'Responsei75n75', 'Responsei75r25', 'Responsei75r75', 'Responsei75s25', 'Responsei75s75', 'Stimulusc25n25', 'Stimulusc25r25', 'Stimulusc25r75', 'Stimulusc25s25', 'Stimulusc25s75', 'Stimulusc75r25', 'Stimulusc75r75', 'Stimulusc75s25', 'Stimulusc75s75', 'Stimulusi25n75', 'Stimulusi25r25', 'Stimulusi25r75', 'Stimulusi25s25', 'Stimulusi25s75', 'Stimulusi75n25', 'Stimulusi75n75', 'Stimulusi75r25', 'Stimulusi75r75', 'Stimulusi75s25', 'Stimulusi75s75']
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 7169 original time points ...
0 bad epochs dropped


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done   4 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-2)]: Done  11 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-2)]: Done  18 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-2)]: Done  27 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-2)]: Done  47 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-2)]: Done  58 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-2)]: Done  71 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-2)]: Done  84 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-2)]: Done  99 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-2)]: Done 114 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-2)]: Done 131 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-2)]: Done 148 tasks      | elapsed:   19.8s


Not setting metadata


[Parallel(n_jobs=-2)]: Done 176 out of 176 | elapsed:   23.0s finished


Responsec25s25: (20, 176, 46, 255)
